In [12]:
import scipy.io.wavfile as wav
import numpy as np
import random
import IPython.display as ipd
import matplotlib.pyplot as plt
import math 
%matplotlib inline
import librosa
import midiutil
import sys
from scipy.interpolate import interp1d



y, sr = librosa.load('vocals.wav')
print(y.shape)
print(sr)
voiced_acc = 0.9
onset_acc = 0.8
frame_length=2048
window_length=1024
hop_length=256
pitch_acc = 0.99
spread = 0.6


f0, voiced_flag, voiced_probs = librosa.pyin(y,librosa.note_to_hz('C2'),librosa.note_to_hz('C7'),sr,frame_length, window_length, hop_length)
#f0 = np.nan_to_num(f0)
#0_ = np.round(librosa.hz_to_midi(f0)).astype(int)
f0_ = np.where(f0_ == -2147483648, 0,f0_)
#f0_new = np.nan_to_num(f0)
onsets = librosa.onset.onset_detect(y, sr=sr, hop_length=hop_length, backtrack=True)
print(f0_[:30])

print(f0_.shape)
print(onsets)

silence = 0
onset = 1
sustain = 2
my_state = silence

last_onset = 0
last_offset = 0
last_midi = 0
output = []
hop_time = hop_length/sr

# for loop inspired by audio_to_mid repo 
# https://github.com/tiagoft/audio_to_midi/blob/1f337672de6942dcd97b9b5b6617f5e84c0e6c5a/audio2midi.py#L145
for i in range(len(f0_)):
        if my_state == silence:
            if int(f0_[i]%2) != 0:
                # Found an onset!
                last_onset = i * hop_time
                last_midi = ((f0_[i]-1)/2)
                my_state = onset


        elif my_state == onset:
            if int(f0_[i]%2) == 0:
                my_state = sustain

        elif my_state == sustain:
            if int(f0_[i]%2) != 0:
                # Found an onset.                
                # Finish last note
                last_offset = i*hop_time
                my_note = [last_onset, last_offset, last_midi]
                output.append(my_note)
                
                # Start new note
                last_onset = i * hop_time
                last_midi = ((f0_[i]-1)/2)
                my_state = onset
            
            elif f0_[i]==0:
                # Found silence. Finish last note.
                last_offset = i*hop_time
                my_note = [last_onset, last_offset, last_midi]
                output.append(my_note)
                my_state = silence


print(output)

MyMIDI = midiutil.MIDIFile(1)
bpm = librosa.beat.tempo(y)[0]
MyMIDI.addTempo(0, 0, bpm)

onsets = np.array([p[0] for p in output])
offsets = np.array([p[1] for p in output])
quarter_note = 60/bpm
ticks_per_quarter = 1024
onsets = onsets / quarter_note
offsets = offsets  / quarter_note


for i in range(len(onsets)):
        print(f0_[i], onsets[i])
        MyMIDI.addNote(0, 0, f0_[i], onsets[i], 1, 100)

with open("output.mid", "wb") as output_file:
        MyMIDI.writeFile(output_file)




[         nan          nan 372.1377786  ... 325.8414615  311.12698372
 311.12698372]
